In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 15 11:01:32 2022

@author: Simone
"""

import numpy as np
import pandas as pd 
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN,KMeans
from sklearn.preprocessing import StandardScaler
import pyproj
import json
from shapely.geometry import Point, mapping
from functools import partial
from shapely.ops import transform
from mpl_toolkits.mplot3d import proj3d
from scipy.spatial.distance import cdist
import seaborn as sns


%matplotlib notebook
df = pd.read_csv("New/datasetItalyComplete.csv", dtype = {"ITTER107":str})
df = df.loc[df['Sesso'] != "totale"]
df = df.loc[df['Stato civile'] != "totale"]

In [ ]:
df['populPerKm2'] = df['Value']/df['Size km2']
df = df.loc[df['populPerKm2'] > 10]#Filter out small villages
df = df.reset_index().drop(["index"],axis =1)#.to_csv("datasetItalyCompleteNoSmallVill.csv", encoding='utf-8',index=False)
df

## Get only Lombardy

In [ ]:
# lombardy = pd.read_excel('New/Elenco-comuni-italiani.xls', sheet_name=0) #reads the first sheet of your excel file
# lombardy = lombardy[lombardy['Denominazione Regione']=='Lombardia'] #Filtering dataframe
# df["ITTER107"] = pd.to_numeric(df["ITTER107"])
# df = pd.merge(df, lombardy,how='inner',left_on=['ITTER107'],right_on=['Codice Comune formato alfanumerico'])
# df.drop(lombardy.columns,axis = 1, inplace = True)
# df

In [ ]:

# heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
# heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
dfSliced = df[['lng','lat','Sesso','Stato civile','AgeGroup','AvgNumberOfComponents','Redditi 2020']]
catColumnsPos = [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('object').columns)]
catColumnsPos += [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('category').columns)]

In [ ]:
#No if kprot
dfSlicedNoCat = pd.get_dummies(dfSliced)
dfSlicedNoCat.describe()

### Normalize

In [ ]:
# from sklearn import preprocessing
# x = dfSliced[["AvgNumberOfComponents","Redditi 2020"]].values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# dfSliced[["AvgNumberOfComponents","Redditi 2020"]] = pd.DataFrame(
#     x_scaled,columns=dfSliced[["AvgNumberOfComponents","Redditi 2020"]].columns,)

### Standardize

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(dfSlicedNoCat[["AvgNumberOfComponents","Redditi 2020"]].values)
scaled_features_df = pd.DataFrame(scaled_features, 
                                  index=dfSlicedNoCat[["AvgNumberOfComponents","Redditi 2020"]].index, 
                                  columns=dfSlicedNoCat[["AvgNumberOfComponents","Redditi 2020"]].columns)
dfSlicedNoCat[["AvgNumberOfComponents","Redditi 2020"]] = scaled_features_df

scaler = StandardScaler()
scaled_features = scaler.fit_transform(dfSliced[["AvgNumberOfComponents","Redditi 2020"]].values)
scaled_features_df = pd.DataFrame(scaled_features, 
                                  index=dfSliced[["AvgNumberOfComponents","Redditi 2020"]].index, 
                                  columns=dfSliced[["AvgNumberOfComponents","Redditi 2020"]].columns)
dfSliced[["AvgNumberOfComponents","Redditi 2020"]] = scaled_features_df

In [ ]:
dfSlicedNoCat

In [ ]:
dfSliced

In [ ]:
# import seaborn as sns

# heatmap = sns.heatmap(dfSlicedNoCat.corr(), vmin=-1, vmax=1, annot=True)
# heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

## Elbow KMeans


In [ ]:
# # Choose optimal K using Elbow method
# distortions = []
# inertias = []

# rangeCl = range(10, 100)

# print(catColumnsPos)
# for cluster in rangeCl:
#     kmeans = KMeans( n_clusters = cluster, init = 'k-means++', random_state = 0)
#     categories = kmeans.fit_predict(dfSlicedNoCat)
#     distortions.append(sum(np.min(cdist(dfSlicedNoCat, kmeans.cluster_centers_,
#                                         'euclidean'), axis=1)) / dfSlicedNoCat.shape[0])
#     inertias.append(kmeans.inertia_)
#     print('Cluster initiation: {}'.format(cluster))

In [ ]:
# #Data viz

# plt.plot(rangeCl, inertias, 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Inertia')
# plt.title('The Elbow Method using Inertia')
# plt.show()

In [ ]:
# plt.plot(rangeCl, distortions, 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method using Distortion')
# plt.show()

### Clustering KMeans

In [ ]:
nClusters = 170 #88 Lombardy #170 Italy #250 too many low silh

kmeans = KMeans( n_clusters = nClusters, init = 'k-means++', random_state = 0)
kMeansCategories = kmeans.fit_predict(dfSlicedNoCat)
df["cluster"] = kMeansCategories

In [ ]:
# df#.to_csv("clusteringItalia.csv", encoding='utf-8',index=False)

## Elbow KPrototypes

In [ ]:
# #Choose optimal K using Elbow method
# cost = []


# print(catColumnsPos)
# for cluster in range(60, 86):
#     try:
#         kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
#         kprototype.fit_predict(dfSliced, categorical = catColumnsPos)
#         cost.append(kprototype.cost_)
#         print('Cluster initiation: {}'.format(cluster))
#     except:
#         break
# # Converting the results into a dataframe and plotting them
# #Data viz

# plt.plot( range(60, 86), cost, 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method using Distortion')
# plt.show()
# plt.savefig('elbow.png', dpi=fig.dpi)

### Clustering KPrototype

In [ ]:
from kmodes.kprototypes import KPrototypes

nClusters = 4 #40

catColumnsPos = [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('object').columns)]
catColumnsPos += [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('category').columns)]
kprototype = KPrototypes(n_jobs = -1, n_clusters = nClusters, init = 'Huang', random_state = 0)
KProtcategories = kprototype.fit_predict(dfSliced, categorical = catColumnsPos)
df["cluster"] = KProtcategories


## SOM

In [ ]:
# from hyperopt import fmin, hp, Trials, tpe, STATUS_OK
# from minisom import MiniSom
# # Setting some parameters in advance
# x = y = 10
# input_len = dfSlicedNoCat.shape[1]
# sigma = 1.0
# learning_rate = 0.5
# iterations = 1000

# space = {
#     'sig': hp.uniform('sig',0.001,5.0),
#     'learning_rate': hp.uniform('learning_rate',0.001,1),
#     'x' :  hp.uniform('x',10,100),
#     'y' :  hp.uniform('y',10,100)
# }

# def opt_map(space):
#     sig = space['sig']
#     learning_rate = space['learning_rate']
#     x = space['x']
#     y = space['y']
#     val = MiniSom(x=int(x), y=int(y), input_len=input_len, sigma=sigma, learning_rate=learning_rate).quantization_error(dfSlicedNoCat.values)
#     #print("Now,the quantization error is {}\n".format(val))
#     return {'loss':val, 'status':STATUS_OK}

# trials = Trials()
# best_params = fmin(fn=opt_map,space=space,algo=tpe.suggest,max_evals=10,trials=trials)
# print("The best sigma value after 50 iterations is {}".format(best_params['sig']))
# print("The best learning_rate after 50 iterations is {}".format(best_params['learning_rate']))
# print("The best x after 50 iterations is {}".format(int(best_params['x'])))
# print("The best y after 50 iterations is {}".format(int(best_params['y'])))

In [ ]:
# from hyperopt import fmin, hp, Trials, tpe, STATUS_OK
# from minisom import MiniSom
# # Setting some parameters in advance
# x = y = 10
# input_len = dfSlicedNoCat.shape[1]
# sigma = 1.0
# learning_rate = 0.5
# iterations = 1000

# space = {
#     'sig': hp.uniform('sig',0.001,5.0),
#     'learning_rate': hp.uniform('learning_rate',0.001,1),
#     'x' :  hp.uniform('x',10,100),
#     'y' :  hp.uniform('y',10,100)
# }

# def opt_map(space):
#     sig = space['sig']
#     learning_rate = space['learning_rate']
#     x = space['x']
#     y = space['y']
#     som = MiniSom(x=int(x), y=int(y), input_len=input_len, sigma=sigma, 
#                   learning_rate=learning_rate)
#     som.train_batch(dfSlicedNoCat.values, 10000)
#     winner_coordinates = np.array([som.winner(x) for x in dfSlicedNoCat.values]).T
#     # with np.ravel_multi_index we convert the bidimensional
#     # coordinates to a monodimensional index
#     SOMCategories = np.ravel_multi_index(winner_coordinates, (int(x),int(y)))
#     val = -silhouette_score(dfSlicedNoCat,SOMCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0)
#     #print("Now,the quantization error is {}\n".format(val))
#     return {'loss':val, 'status':STATUS_OK}

# trials = Trials()
# best_params = fmin(fn=opt_map,space=space,algo=tpe.suggest,max_evals=10,trials=trials)
# print("The best sigma value after 50 iterations is {}".format(best_params['sig']))
# print("The best learning_rate after 50 iterations is {}".format(best_params['learning_rate']))
# print("The best x after 50 iterations is {}".format(int(best_params['x'])))
# print("The best y after 50 iterations is {}".format(int(best_params['y'])))

In [ ]:
from minisom import MiniSom

# With quantization_error
som_shape = (41,42)
lr = 0.9310156988049898
sig = 1.590577785080987

# With topographic error
# som_shape = (35,10)
# lr = 0.19228430625877874
# sig = 2.0404878138221725

# With silhouette
# som_shape = (61,33)
# lr = 0.4207120861264718
# sig = 4.443618124736826

# With silhouette more iterations
# som_shape = (14,91)
# lr = 0.45328188042355616
# sig = 2.0044976649797683

som = MiniSom(x=som_shape[0], y=som_shape[1], input_len=dfSlicedNoCat.shape[1], learning_rate = lr, sigma = sig)
som.train_batch(dfSlicedNoCat.values, 100000,  verbose=True)
# each neuron represents a cluster
winner_coordinates = np.array([som.winner(x) for x in dfSlicedNoCat.values]).T
# with np.ravel_multi_index we convert the bidimensional
# coordinates to a monodimensional index
SOMCategories = np.ravel_multi_index(winner_coordinates, som_shape)
df["cluster"] = SOMCategories
nClusters = max(SOMCategories)

In [ ]:
from sklearn.metrics.cluster import silhouette_score

silhouette_score(dfSlicedNoCat,SOMCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0)


## Fuzzy C-Means

In [ ]:
from pyclustering.cluster.fcm import fcm
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from sklearn.metrics.cluster import silhouette_score

# Choose optimal K using Elbow method
silhouette = []

rangeCl = range(10, 12)

for cluster in rangeCl:
    initial_centers = kmeans_plusplus_initializer(dfSlicedNoCat, cluster, kmeans_plusplus_initializer.FARTHEST_CENTER_CANDIDATE).initialize()
    fcm_instance = fcm(dfSlicedNoCat.values, initial_centers)
    fcm_instance.process()
    clusters = fcm_instance.get_clusters()
    df["cluster"] = 1
    i=0
    for c in clusters:
        df.iloc[c,df.columns.get_loc('cluster')] =i
        i = i+1
    FCMCategories = df["cluster"].values
    
    silhouette.append(silhouette_score(dfSlicedNoCat,FCMCategories,
                                               sample_size = int(len(dfSlicedNoCat)/5),random_state = 0))
    print('Cluster initiation: {}'.format(cluster))
#Data viz

plt.plot(rangeCl, silhouette, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Silhouette score')
plt.title('The Elbow Method using Silhouette')
plt.show()

In [ ]:
from pyclustering.cluster.fcm import fcm
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer

amount_initial_centers = 20

# initialize
initial_centers = kmeans_plusplus_initializer(dfSlicedNoCat, amount_initial_centers, kmeans_plusplus_initializer.FARTHEST_CENTER_CANDIDATE).initialize()
 
# create instance of Fuzzy C-Means algorithm
fcm_instance = fcm(dfSlicedNoCat.values, initial_centers)
 
# run cluster analysis and obtain results
fcm_instance.process()
clusters = fcm_instance.get_clusters()
centers = fcm_instance.get_centers()

print( len(centers))
nClusters = len(clusters)
df["cluster"] = 1
i=0
for c in clusters:
    df.iloc[c,df.columns.get_loc('cluster')] =i
    i = i+1
FCMCategories = df["cluster"].values

## HDBSCAN

In [ ]:
# from sklearn.cluster import OPTICS
# #Minpts = domain knowledge
# optics = OPTICS(min_samples=20, metric='euclidean' )
# OPTICSCategories = optics.fit_predict(dfSlicedNoCat)
# df["cluster"] = OPTICSCategories
# nClusters = max(OPTICSCategories)

In [ ]:
# len(np.unique(OPTICSCategories))

In [ ]:
# !pip install hdbscan

In [ ]:
from hyperopt import fmin, hp, Trials, tpe, STATUS_OK
import hdbscan

# specify parameters and distributions to sample from
space = {'min_samples':  hp.uniform('min_samples',10,100),
              'min_cluster_size': hp.uniform('min_cluster_size',100,600),  
              'cluster_selection_method' : hp.choice('cluster_selection_method', ['eom','leaf']),
              'metric' : hp.choice('metric', ['euclidean','manhattan']),
             }

def opt_map(space):
    min_samples = space['min_samples']
    min_cluster_size = space['min_cluster_size']
    cluster_selection_method = space['cluster_selection_method']
    metric = space['metric']
    
    hdb = hdbscan.HDBSCAN(min_samples=int(min_samples),min_cluster_size=int(min_cluster_size),
                         cluster_selection_method=cluster_selection_method,metric=metric,
                         algorithm="best",gen_min_span_tree=True).fit(dfSlicedNoCat)
    
    val = -hdb.relative_validity_
    return {'loss':val, 'status':STATUS_OK}

trials = Trials()
best_params = fmin(fn=opt_map,space=space,algo=tpe.suggest,max_evals=4,trials=trials)
print("The best min_samples value after 50 iterations is {}".format(int(best_params['min_samples'])))
print("The best min_cluster_size after 50 iterations is {}".format(int(best_params['min_cluster_size'])))
print("The best cluster_selection_method after 50 iterations is {}".format(best_params['cluster_selection_method']))
print("The best metric after 50 iterations is {}".format(best_params['metric']))



In [ ]:
import hdbscan

hdbCategories = hdbscan.HDBSCAN(min_samples=22,min_cluster_size=102,
                         cluster_selection_method='leaf',metric='euclidean',
                         algorithm="best",gen_min_span_tree=True).fit_predict(dfSlicedNoCat)
hdbCategories
df["cluster"] = hdbCategories
nClusters = max(hdbCategories)
print(f"DBCV score {hdbscan.validity.validity_index(dfSlicedNoCat.values,hdbCategories, metric = 'euclidean')}")
print(len(np.unique(hdbCategories)))

In [ ]:
hdbscan.validity.validity_index(dfSlicedNoCat.values,hdbCategories, metric = 'euclidean')

In [ ]:
# 16 272 eom man -->0.219
# 86 577 eom man --> 0.188
# Provo ancora con relative, poi con validity

## Xmeans

In [ ]:
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer

# Prepare initial centers - amount of initial centers defines amount of clusters from which X-Means will
# start analysis.
amount_initial_centers = 10
initial_centers = kmeans_plusplus_initializer(dfSlicedNoCat, amount_initial_centers).initialize()
amount_max_centers = 3000

xmeans_instance = xmeans(dfSlicedNoCat, initial_centers,amount_max_centers, tolerance = 0.002) #,tolerance = 0.001
xmeans_instance.process()

# Extract clustering results: clusters and their centers
clusters = xmeans_instance.get_clusters() #cluster[0] are index of points attached to center 0
centers = xmeans_instance.get_centers()

print( len(centers))
nClusters = len(clusters)
df["cluster"] = 1
i=0
for c in clusters:
    df.iloc[c,df.columns.get_loc('cluster')] =i
    i = i+1
xmeansCategories = df["cluster"].values
#2380 with default tolerance

## Analyze clusters
##### By changing cluster number and see which people comprises

In [ ]:
# for col in df.columns:
#     if df.columns.get_loc(col) < 4:
#         print(df.loc[df["cluster"] == selectedCluster][col].value_counts())

In [ ]:
for i in range(0,nClusters):#For each cluster
    print("Cluster #" + str(i))
    for col in df.columns: #For each "important" column
        if (col not in ["Value","Territorio","lat","lng","Size km2","populPerKm2","cluster","ITTER107"]):
            print(df.loc[df["cluster"] == i][col].astype(str).value_counts())
    print()#Whitespace

In [ ]:
clusterNames = []
for i in range(0,nClusters):#For each cluster
    clusterName = ""
    for col in df.columns: #For each "important" column
        if (col not in ["Value","Territorio","lat","lng","Size km2","populPerKm2","cluster","ITTER107"]):
            if col == "Redditi 2020":
                clusterName += "Reddito medio " + str(int(df.loc[df["cluster"] == i][col].mean())) + " "
            elif col == "AvgNumberOfComponents":
                clusterName += "Numero medio di componenti " + str(round(df.loc[df["cluster"] == i][col].mean(),2)) + " "
            else:
                clusterName += str(df.loc[df["cluster"] == i][col].value_counts().idxmax()) + " "
    clusterNames.append(clusterName)

In [ ]:
clusterNames

In [ ]:
clusterNames = []
for i in range(0,max(df.cluster)):#For each cluster
    clusterName = ""
    for col in df.columns: #For each "important" column
        if (col not in ["Value","Territorio","lat","lng","Size km2","populPerKm2","cluster","ITTER107"]):
            if col == "Redditi 2020":
                clusterName += str(int(df.loc[df["cluster"] == i][col].mean())) + " "
            elif col == "AvgNumberOfComponents":
                clusterName += str(round(df.loc[df["cluster"] == i][col].mean(),2)) + " "
            else:
                clusterName += str(df.loc[df["cluster"] == i][col].value_counts().idxmax()) + " "
    clusterName += str(i)
    clusterNames.append(clusterName)

splitted_search=[x.split(" ") for x in clusterNames]
clusterTypes = pd.DataFrame(splitted_search, columns =['Gender', 'MaritalStatus', 'AgeGroup',
                                                            'AvgIncome','AvgNumComp','cluster'])
clusterTypes["AvgIncome"] = clusterTypes["AvgIncome"].astype(int)
clusterTypes["cluster"] = clusterTypes["cluster"].astype(int)

In [ ]:
# curr = clusterTypes.loc[(clusterTypes["AgeGroup"] == ageGroup) & 
#                                         (clusterTypes["Gender"] == gender) & 
#                                         (clusterTypes["MaritalStatus"] == maritalStatus)]
# income = 5 
# # min(curr.values, key=lambda x:print(x))
# def takeClosest(curr, targetIncome):
#     closestIncome = curr.iloc[:1]["AvgIncome"].values[0]
#     for i in range(1, curr.shape[0]):    
#         currInc = curr.iloc[i:i+1]["AvgIncome"].values[0]
#         if abs(currInc - targetIncome) < abs(closestIncome - targetIncome):
#             closestIncome = currInc
#     return closestIncome

# df.loc[df["cluster"] == curr.loc[curr["AvgIncome"] == takeClosest(curr,income)]["cluster"].values[0]]
# # curr.loc[curr["AvgIncome"] == takeClosest(curr,income)]["cluster"].values[0]

## Visualization

In [ ]:
# # Extract the data we're interested in
# lat = df['lat'].values
# lon = df['lng'].values
# population = df['Value'].values
# populationsqm = df['populPerKm2'].values
# #area = cities['area_total_km2'].values
selectedCluster = 84
lat = df.loc[df["cluster"] == selectedCluster]['lat'].values
lon = df.loc[df["cluster"] == selectedCluster]['lng'].values
population = df.loc[df["cluster"] == selectedCluster]['Value'].values
populationsqm = df.loc[df["cluster"] == selectedCluster]['populPerKm2'].values
area = df.loc[df["cluster"] == selectedCluster]['Size km2'].values

## Useful functions

In [ ]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def getxyz(event):
    s = ax.format_coord(event.xdata, event.ydata)
    try:
        out = [float(x.split('=')[1].strip()) for x in s.split(',')]
    except:
        out = None
    return out

def onclick(event): #Need to zoom  a bit for it to work (????)
    out = getxyz(event)
    if out != None:
        az = ax.azim
        el = ax.elev
        if (abs(el) <= 2): #If elevation 0
            if (abs(az) <= 2 or abs(abs(az) -180) <= 2): # get z and not x
                zPt = out[2]
                latPt = out[1]
                closest = closest_point((zPt/100,latPt), list(zip(populationsqm/100, lat)))#Get closest point from picked
                tx = '%s' % (df.loc[(df["populPerKm2"] >= (closest[0]*100-0.1)) & 
                                    (df["populPerKm2"] <= (closest[0]*100+0.1))&
                                    (df["cluster"] == selectedCluster)&
                                    (df["lat"] == closest[1])]["Territorio"].values[0])
                ax.set_title(tx)
            elif (abs(az)-90 <= 2):# get z and not y
                zPt = out[2]
                lonPt = out[0]
                tx = out[0]
                closest = closest_point((zPt/100,lonPt), list(zip(populationsqm/100, lon)))#Get closest point from picked
                tx = '%s' % (df.loc[(df["populPerKm2"] >= (closest[0]*100-0.1)) & 
                                    (df["populPerKm2"] <= (closest[0]*100+0.1))&
                                    (df["cluster"] == selectedCluster)&
                                    (df["lng"] == closest[1])]["Territorio"].values[0])
                ax.set_title(tx)
            else:
                ax.set_title("Please adjust azimuth")
        elif (abs(el-90) <= 2):
            lonPt = out[0]#llcrnrlon+(urcrnrlon-llcrnrlon)/2 + event.xdata*((urcrnrlon-llcrnrlon)/2)/0.05481
            latPt = out[1]#llcrnrlat+(urcrnrlat-llcrnrlat)/2 + event.ydata*((urcrnrlat-llcrnrlat)/2)/0.05481
            closest = closest_point((lonPt,latPt), list(zip(lon, lat)))#Get closest point from picked
            tx = '%s' % (df.loc[(df["lng"] == closest[0])&
                                (df["cluster"] == selectedCluster)&
                                (df["lat"] == closest[1])]["Territorio"].values[0])
            ax.set_title(tx)
        else:
            ax.set_title("Please adjust elevation")

In [ ]:
def kmRadius(c): #Define area of attractiveness
    point = Point(c)
    
    local_azimuthal_projection = f"+proj=aeqd +R=6371000 +units=m +lat_0={point.y} +lon_0={point.x}"

    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
        pyproj.Proj(local_azimuthal_projection),
    )

    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
    )

    point_transformed = transform(wgs84_to_aeqd, point)

    buffer = point_transformed.buffer(20_000)#Meters

    buffer_wgs84 = transform(aeqd_to_wgs84, buffer)
    dic = mapping(buffer_wgs84)
    return np.asarray(dic['coordinates'])[0]

##Mean based on: municipalities with more target population per sqmt influence more (can be **2 or **3)
##Remove far away points from calculation
def calculateWeightedMean(lat, lon, populationsqm):
    avLat = np.average(a=lat[clf > 0], weights=populationsqm[clf > 0]**3)
    avLon = np.average(a=lon[clf > 0], weights=populationsqm[clf > 0]**3)
    return avLon,avLat


## 3D map Lombardy

In [ ]:
# for i in range (0, 4):
#     # Extract the data we're interested in
#     lat = df.loc[df["cluster"] == i]['lat'].values
#     lon = df.loc[df["cluster"] == i]['lng'].values
#     population = df.loc[df["cluster"] == i]['Value'].values
#     populationsqm = df.loc[df["cluster"] == i]['populPerKm2'].values
#     area = df.loc[df["cluster"] == i]['Size km2'].values
fig = plt.figure(0)
ax = fig.add_subplot(projection='3d')
# print(clusterNames[selectedCluster])

llcrnrlat = 35.5#44
urcrnrlat = 47
llcrnrlon = 7#7
urcrnrlon = 18.5#12
# Create a basemap instance that draws the Earth layer
bm = Basemap(llcrnrlat=llcrnrlat, llcrnrlon=llcrnrlon, 
            urcrnrlat=urcrnrlat, urcrnrlon=urcrnrlon,
            projection='cyl', resolution='l', fix_aspect=False, ax=ax)
# Add Basemap to the figure
ax.add_collection3d(bm.drawcoastlines(linewidth=0.25))
ax.add_collection3d(bm.drawcountries(linewidth=0.35))
ax.view_init(azim=-90, elev=90)
ax.set_xlabel('Longitude (°E)', labelpad=20)
ax.set_ylabel('Latitude (°N)', labelpad=20)
ax.set_zlabel('Population per sqkm', labelpad=20)
ax.set_zlim(0., 400)

colormap = np.arange(0,15)

from sklearn.ensemble import IsolationForest

clf = IsolationForest( random_state=0).fit_predict(np.array((lon,lat)).T)#Schema a blochi pptx
print(np.array((lon,lat)).T)

lonC, latC = calculateWeightedMean(lat, lon, populationsqm)
p = ax.scatter(lon, lat, (populationsqm), picker = True, s = area,c=colormap[clf])#Points # c=colormap[categories]
centers = ax.scatter(xs=lonC, ys=latC, edgecolor='black', c='white', marker = '*')#Centers

circle = kmRadius((lonC,latC))
r = ax.scatter(xs=circle[:,0], ys=circle[:,1], c='black', marker = '1')#Radius

closest = closest_point((lonC,latC), list(zip(lon, lat)))#Get closest point from picked

tx = 'Center is located in: %s' % (df.loc[(df["lng"] == closest[0])&
                                          (df["cluster"] == selectedCluster)&
                                          (df["lat"] == closest[1])]["Territorio"].values[0])
ax.set_title(tx)
print(tx)
cid = fig.canvas.mpl_connect('button_press_event', onclick)#Get mouse click

fig.show
print(clusterNames[selectedCluster])

In [ ]:
df.loc[(df["cluster"] == selectedCluster) & (df["Territorio"] == "Cusano Milanino")]

In [ ]:
# df.loc[(df["cluster"] == selectedCluster) & (df["Territorio"] == "Lecco")]

## PCA no categorical

### Graph num of components

In [ ]:
from sklearn.decomposition import PCA
%matplotlib inline
pca = PCA(n_components=21, random_state = 0)
principalComponents = pca.fit_transform(dfSlicedNoCat)
principalDf = pd.DataFrame(data = principalComponents)
print("Variance remained", np.sum(pca.explained_variance_ratio_))
#% of info contained with n components
plt.figure(figsize = (8,8))

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of components")
plt.ylabel("Explained variance")
plt.grid()

plt.savefig("Elbow_PCA.png")
# plt.show()

### 2 components

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state = 0)
principalComponents = pca.fit_transform(dfSlicedNoCat)
principalDf = pd.DataFrame(data = principalComponents)
print("Info remained with 2 components:", np.sum(pca.explained_variance_ratio_))
#% of info contained with n components

In [ ]:
principalDf

In [ ]:
plt.figure(figsize = (8,8))
sns.scatterplot(x = principalComponents[:,0], y = principalComponents[:,1]) 
plt.xlabel('Principal Component 1', fontsize = 15)
plt.ylabel('Principal Component 2', fontsize = 15)
plt.title('2 component PCA', fontsize = 20)

### 3 components

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3, random_state = 0)
principalComponents = pca.fit_transform(dfSlicedNoCat)
principalDf = pd.DataFrame(data = principalComponents)
print("Info remained with 3 components:", np.sum(pca.explained_variance_ratio_))
#% of info contained with n components

In [ ]:
Xax = principalComponents[:,0]
Yax = principalComponents[:,1]
Zax = principalComponents[:,2]

cdict = {0:'red',1:'green'}
labl = {0:'Malignant',1:'Benign'}
marker = {0:'*',1:'o'}
alpha = {0:.3, 1:.5}

fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111, projection='3d')

fig.patch.set_facecolor('white')
ax.scatter(Xax, Yax, Zax)
# for loop ends
ax.set_xlabel("First Principal Component", fontsize=14)
ax.set_ylabel("Second Principal Component", fontsize=14)
ax.set_zlabel("Third Principal Component", fontsize=14)

plt.show()

## FAMD (PCA  + categorical)

### Graph num of components

In [ ]:
import prince

famd = prince.FAMD(
     n_components=18,
     n_iter = 3,
     engine='auto',
     random_state=0
)
famd = famd.fit(dfSliced)
famdTr = famd.transform(dfSliced)
print("Variance remained",np.sum(famd.explained_inertia_))
plt.figure(figsize = (8,8))

plt.plot(np.cumsum(famd.explained_inertia_))
plt.xlabel("Number of components")
plt.ylabel("Explained variance")
plt.grid()

# plt.savefig("Elbow_FAMD.png")

### 2 components

In [ ]:
famd = prince.FAMD(
     n_components=2,
     n_iter = 3,
     engine='auto',
     random_state=0
)
famd = famd.fit(dfSliced)
famdTr = famd.transform(dfSliced)
print("Variance remained",np.sum(famd.explained_inertia_))

ax = famd.plot_row_coordinates(
     X = dfSliced,
     ax=None,
     figsize=(6, 6),
     x_component=0,
     y_component=1,
     ellipse_outline=True,
     ellipse_fill=True,
     show_points=True
)
# ax.get_figure().savefig('famd_row_coordinates.png')


## Compare clustering results

In [ ]:
# CHScore = []# Higher better
# DBScore = []# 0 best
score = []# 1 best, -1 worst

In [ ]:
# from sklearn.metrics.cluster import homogeneity_score
# from sklearn.metrics.cluster import adjusted_mutual_info_score
# from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import calinski_harabasz_score
from sklearn.metrics.cluster import davies_bouldin_score
from sklearn.metrics.cluster import silhouette_score

# homogeneity_score(kMeansCategories,Y)
# adjusted_mutual_info_score(kMeansCategories, [0, 0, 1, 1])
# adjusted_rand_score(kMeansCategories, [0, 0, 1, 1])
# calinski_harabasz_score (dfSlicedNoCat,kMeansCategories)
# davies_bouldin_score(dfSlicedNoCat,kMeansCategories)
# silhouette_score(dfSlicedNoCat,kMeansCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0) 

# score.append(["KMeans with Elbow",
#               calinski_harabasz_score (dfSlicedNoCat,kMeansCategories),
#               davies_bouldin_score(dfSlicedNoCat,kMeansCategories),
#               silhouette_score(dfSlicedNoCat,kMeansCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0),
#               len(np.unique(kMeansCategories))])

# score.append(["KPrototype with Elbow",
#                 calinski_harabasz_score (dfSlicedNoCat,KProtcategories),
#                 davies_bouldin_score(dfSlicedNoCat,KProtcategories),
#               silhouette_score(dfSlicedNoCat,KProtcategories,sample_size = int(len(dfSliced)/5),random_state = 0),
#             len(np.unique(KProtcategories))])

# score.append(["Xmeans",calinski_harabasz_score (dfSlicedNoCat,xmeansCategories),
#               davies_bouldin_score(dfSlicedNoCat,xmeansCategories),
#               silhouette_score(dfSlicedNoCat,xmeansCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0),
#              len(np.unique(xmeansCategories))])

# score.append(["SOM",calinski_harabasz_score (dfSlicedNoCat,SOMCategories),
#               davies_bouldin_score(dfSlicedNoCat,SOMCategories),
#               silhouette_score(dfSlicedNoCat,SOMCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0),
#             len(np.unique(SOMCategories))])

# score.append(["HDBSCAN",calinski_harabasz_score (dfSlicedNoCat,hdbCategories),
#               davies_bouldin_score(dfSlicedNoCat,hdbCategories),
#               hdbscan.validity.validity_index(dfSlicedNoCat.values,hdbCategories, metric = 'euclidean'),
#              len(np.unique(hdbCategories))])

score.append(["FCM",calinski_harabasz_score (dfSlicedNoCat,FCMCategories),
              davies_bouldin_score(dfSlicedNoCat,FCMCategories),
              silhouette_score(dfSlicedNoCat,FCMCategories,sample_size = int(len(dfSlicedNoCat)/5),random_state = 0),
             len(np.unique(FCMCategories))])


In [ ]:
pd.DataFrame(score, columns = ['Algorithm', 'CH Value(higher better)','DB Value(0 best)', 'Silhouette value(1 best)', 'Number of clusters'])